<h2>Nettoyage des données inutilisables</h2>

In [31]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [32]:
df = pd.read_excel('IM2S_GHS_RSS_Bloc_2019_2022_anonyme.xlsx')
pd.options.display.max_columns = 33
pd.options.display.max_rows = 50

#renomme les colonnes  
df.columns = ['NoCas', 'DateEntry', 'DateExit', 'DateBirth', 'Sex',
       'CIMDiagPr', 'CimAssoc1', 'CimAssoc2', 'CimAssoc3',
       'CimAssoc4', 'CimAssoc5', 'CCAM1', 'CCAM2', 'CCAM3', 'CCAM4',
       'GHMCode', 'Colonne2', 'Colonne1', 'Molon', 'Anesthesiste',
       'Chirurgien', 'Year', 'IDPatient', 'DateInter', 'NomPatient',
       'HourEntrySSPI',
       'HourEntryBLOC', 'HourIncision ',
       'HourExitBLOC', 'AnesthType',
       'AnesthLoco_reg', 'TimeHospit', 'Interv']

#on enlève les colonnes inutiles 
df = df.drop(columns=['Colonne2', 'Colonne1', 'Molon', 'NomPatient'])
df.shape

(14649, 29)

In [33]:
#on enlève les opérations esseulées (mal renseignées par les médecins) 
df = df.groupby('Interv').filter(lambda x : len(x)>8)
#et celle dont le chir n'est pas renseigné 
df = df.dropna(subset = ['Chirurgien', 'Interv']).reset_index(drop=True)
df.shape

(12976, 29)

In [34]:
#Création d'une colonne du temps au bloc 
IntervTime = []
for i in range (len(df)): 
    dtExit = df["HourExitBLOC"].iloc[i]
    dtEntry = df["HourEntryBLOC"].iloc[i]
    intervTime = dtExit.hour-dtEntry.hour + (dtExit.minute-dtEntry.minute)/60
    IntervTime.append(intervTime)

df.insert(25, "IntervTime", IntervTime)

#on enlève les interventions de temps nul
df = df.drop(df[(df["IntervTime"]==0)].index).reset_index(drop=True)
df.shape

(12913, 30)

In [35]:
#Création d'une colonne de temps en SSPI d'entrée 

TimeSSPI = []
for i in range (len(df)): 
        dtSSPI = df["HourEntrySSPI"].iloc[i]
        dtBLOC = df["HourEntryBLOC"].iloc[i]
        sspiTime = dtBLOC.hour-dtSSPI.hour + (dtBLOC.minute-dtSSPI.minute)/60
        if sspiTime == 0 : 
                sspiTime = np.nan
        TimeSSPI.append(sspiTime)

df.insert(25,"SSPITime", TimeSSPI)
        
df.shape

(12913, 31)

<h2>Études Statistiques</h2>

In [36]:
#stat sur les temps d'opération et d'hospitalisation en fonction de l'intervention 
meanIntervTime = df.groupby('Interv')["IntervTime"].mean()
stdIntervTime = df.groupby('Interv')["IntervTime"].std()
meanTimeH = df.groupby('Interv')["TimeHospit"].mean()
stdTimeH = df.groupby('Interv')["TimeHospit"].std()
meanSSPITime = df.groupby('Interv')["SSPITime"].mean() 
stdSSPITime = df.groupby('Interv')["SSPITime"].std()


stat = pd.DataFrame({'meanHospit':meanTimeH, 
                     'stdHopsit':stdTimeH, 
                     'meanInterv':meanIntervTime, 
                     'stdInterv':stdIntervTime,
                    'meanSSPI' : meanSSPITime,
                    'stdSSPI': stdSSPITime})

stat

,meanHospit,stdHopsit,meanInterv,stdInterv,meanSSPI,stdSSPI
Interv,,,,,,
Ablation Broche,1.015385,0.124035,0.585128,0.287834,0.494792,0.350911
Ablation Corps Etranger,1.138889,0.424451,0.799537,0.281901,0.569444,0.294352
Ablation de materiel d Osteosynthese Complexe,1.345794,0.778276,1.133801,0.567191,0.566199,0.365403
Ablation de materiel d Osteosynthese Simple,1.121277,0.369657,0.778511,0.342966,0.544007,0.370506
Arthrodese Arriere Pied,2.326531,1.264709,2.146259,0.529918,0.772109,0.506044
Arthrodese Avant Pied,1.727273,1.009050,2.125758,0.639460,0.681818,0.240191
Arthrodese Cheville,2.583333,1.348644,2.261806,0.664171,0.732639,0.388838
Arthrodese Main,1.222222,0.440959,0.907407,0.249877,0.792593,0.419913
Arthroscopie Cheville,1.239437,0.640585,1.578756,0.593845,0.623759,0.345069
